In [5]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import os


# ==========================================
# SRM CHECK (ROBUST VERSION)
# ==========================================
def check_srm(df, variant_col="variant", alpha=0.05):
    counts = df[variant_col].value_counts()
    
    observed = counts.values.astype(float)
    expected = np.repeat(observed.sum() / len(observed), len(observed))
    
    chi2, p_value = chisquare(f_obs=observed, f_exp=expected)
    
    return "PASS" if p_value > alpha else "FAIL"


# ==========================================
# BALANCE CHECK
# ==========================================
def check_balance(df, variant_col="variant", tolerance=0.05):
    counts = df[variant_col].value_counts(normalize=True)
    
    if len(counts) < 2:
        return "Unknown"
    
    diff = counts.max() - counts.min()
    
    return "Good" if diff < tolerance else "Imbalanced"


# ==========================================
# TEMPORAL STABILITY CHECK
# ==========================================
def check_temporal_stability(df, date_col="timestamp", variant_col="variant"):
    
    if date_col not in df.columns:
        return "Unknown"
    
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    
    daily = df.groupby([df[date_col].dt.date, variant_col]).size().unstack(fill_value=0)
    
    if daily.shape[0] < 2:
        return "Unknown"
    
    daily_ratio = daily.div(daily.sum(axis=1), axis=0)
    variation = daily_ratio.std().mean()
    
    return "Stable" if variation < 0.05 else "Unstable"


# ==========================================
# FULL VALIDATION FUNCTION
# ==========================================
def run_validation(file_path, test_name):
    
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        return {
            "Test": test_name,
            "N": 0,
            "SRM": "File Error",
            "Balance": "-",
            "Temporal": "-",
            "Valid": "NO"
        }
    
    if "variant" not in df.columns:
        return {
            "Test": test_name,
            "N": len(df),
            "SRM": "No variant col",
            "Balance": "-",
            "Temporal": "-",
            "Valid": "NO"
        }
    
    N = len(df)
    srm_status = check_srm(df)
    balance_status = check_balance(df)
    temporal_status = check_temporal_stability(df)
    
    valid = "YES" if (srm_status == "PASS" and balance_status == "Good") else "NO"
    
    return {
        "Test": test_name,
        "N": N,
        "SRM": srm_status,
        "Balance": balance_status,
        "Temporal": temporal_status,
        "Valid": valid
    }


# ==========================================
# RUN ALL TESTS
# ==========================================

BASE_PATH = "/Users/irpanpilihanrambe/Downloads/DATA SET PROJECT DEC/raw/"

tests = {
    "Test 1: Menu Design": BASE_PATH + "test1_menu.csv",
    "Test 2: Novelty Slider": BASE_PATH + "test2_novelty_slider.csv",
    "Test 3: Product Sliders": BASE_PATH + "test3_product_sliders.csv",
    "Test 4: Customer Reviews": BASE_PATH + "test4_reviews.csv",
    "Test 5: Search Engine": BASE_PATH + "test5_search_engine.csv",
}

results = []

for name, path in tests.items():
    results.append(run_validation(path, name))

validation_df = pd.DataFrame(results)

validation_df


,Test,N,SRM,Balance,Temporal,Valid
0,Test 1: Menu Design,7000,PASS,Good,Stable,YES
1,Test 2: Novelty Slider,16000,PASS,Good,Stable,YES
2,Test 3: Product Sliders,18000,PASS,Good,Stable,YES
3,Test 4: Customer Reviews,42000,PASS,Good,Stable,YES
4,Test 5: Search Engine,19000,PASS,Good,Stable,YES


In [1]:
import os
os.getcwd()

'/Users/irpanpilihanrambe/Downloads/DATA SET PROJECT DEC/raw/validation_test'